In [1]:
# Set up the workspace
using SumOfSquares, JuMP, PolyJuMP, DynamicPolynomials, MultivariatePolynomials
using Mosek#, CSDP#, SCS
using Plots
gr()

include("../src/NormalSoS.jl")
using NormalSoS

include("../src/MinimumActionPath.jl")
using MAP

include("../src/GeomMinActPath.jl")
using gMAM

INFO: Recompiling stale cache file /home/rowan/.julia/lib/v0.6/Optim.ji for module Optim.


# Nonlinear Examples

We will examine some nonlinear examples on which the SOS decomposition method is applied.

In [2]:
# A 1D system with a quartic potential
@polyvar x;    x = [x];
F1(X::Vector) = [X[1] - X[1]^3 + 0.1];
f1 = F1(x);
@time U1 = NormalSoS.normdecomp(f1,x, MosekSolver(),1)
NormalSoS.checknorm(f1,U1,x)

Chosen basis as:
DynamicPolynomials.Monomial{true}[x^4, x^3, x^2, x, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 61              
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization pro

status = :Stall
bnd = DynamicPolynomials.Monomial{true}[x^4]
getvalue(ϵ) = [0.249961]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 69              
  Cones                  : 0               
  Scalar variables       : 7               
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic op

 30.573555 seconds (11.39 M allocations: 601.338 MiB, 1.93% gc time)


2.5271169070811736e-5

In [3]:
@show(U1);

U1 = 0.24998873431572866x^4 - 0.4999979210237132x^2 - 0.0999986771350169x + 28657.82240539591


In [3]:
# A 2D quartic system from Zhou et al (2012)
@polyvar x[1:2]
F2(X::Vector) = [-1.0 + 9.0X[1] - 2.0X[1]^3 + 9.0X[2] - 2.0X[2]^3;
      1.0 - 11.0X[1] + 2.0X[1]^3 + 11.0X[2] - 2.0X[2]^3];
f2 = F2(x);
@time Ueg2 = NormalSoS.normdecomp(f2,x, MosekSolver(),1)

Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^4, x1^3x2, x1^2x2^2, x1x2^3, x2^4, x1^3, x1^2x2, x1x2^2, x2^3, x1^2, x1x2, x2^2, x1, x2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 594             
  Cones                  : 0               
  Scalar variables       : 18              
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        :

x1^4 + x1^2x2^2 + x2^4
getvalue(ϵ) = [0.220945, 0.560384, 0.214877]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 621             
  Cones                  : 0               
  Scalar variables       : 17              
  Matrix variables       : 3               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem

0.5000000043330644x1^4 + 0.5000000078114762x2^4 - 4.999999972977872x1^2 + 0.999999824171771x1x2 - 4.999999897900875x2^2 + 0.9999999600165587x1 + 428.8074466955952

## Maier-Stein model

We now look at the widely studied Maier-Stein model which may offer either a pure gradient system or one for which an orthogonal decomposition does not exist.

In [40]:
# The Maier-Stein model
γ = 1.0;    μ = 2.5;
@polyvar x[1:2]
F(X::Vector) = [X[1] - X[1]^3 - γ*X[1]X[2]^2;
                 -μ*(X[1]^2 + 1)X[2]];
f = F(x);
V = -0.5*x[1]^2 + 0.25*x[1]^4 + 0.5γ*x[2]^2 + 0.5γ*x[1]^2*x[2]^2;
@time U = NormalSoS.normdecomp(f,x, MosekSolver(),2)
# NormalSoS.checknorm(f3,U3,x)

fg = -differentiate(U,x)
Fg(X::Vector) = [Float64(subs(fg[1],x[1]=>X[1], x[2]=>X[2]));
                 Float64(subs(fg[2],x[1]=>X[1], x[2]=>X[2]))];
fc = f + differentiate(U,x)
Fc(X::Vector) = [Float64(subs(fc[1],x[1]=>X[1], x[2]=>X[2]));
                 Float64(subs(fc[2],x[1]=>X[1], x[2]=>X[2]))];
U

Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^4, x1^3x2, x1^2x2^2, x1^3, x1^2x2, x1x2^2, x1^2, x1x2, x2^2, x1, x2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 544             
  Cones                  : 0               
  Scalar variables       : 15              
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  

MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(544) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(545) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(546) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(548) of matrix 'A'.
MOSEK warning 705: #6 (nearly) zero elements are specified in sparse row ''(549) of matrix 'A'.
MOSEK warning 705: #5 (nearly) zero elements are specified in sparse row ''(550) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(551) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(552) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(553) of matrix 'A'.
Warning number 705 is disabled.
Problem
  Name                   :                 
  Objective sense        : min             
  Type  

0.24955503412870406x1^4 + 0.5109046760385934x1^2x2^2 - 0.4991115420103632x1^2 + 1.2496491042573663x2^2 + 2014.8422857483358

In [44]:
# Obtain the minimum action path
x₀ = [-1.0 0.0];  # Start point (row vector)
xₑ = [-0.5 0.5];
Tspan = 10.0;  # Time span
N = 400;
φ₀ = MAP.makepath(x₀,xₑ,N);

# for ii=1:N-2
#     φ₀[ii,2] = 0.1*randn();
# end

σ = 1.0;
G(X::Vector) = σ*[1.0; 1.0];

# Use Optim to optimise over path φ
resObj = MAP.optimalpath(F,G, x₀,xₑ, Tspan,N, φ₀);
res = [x₀; Optim.minimizer(resObj); xₑ];

Optimisation for T=10.00 gives S=1.08
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=1.08
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=1.08
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=1.08
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=1.08
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=1.08
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=1.08
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=1.08
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=1.08
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=1.08
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=1.08
Optimisation not converged


In [45]:
# Find the SOS-predicted minimum action path and compare ΔU with gMAM
dt = 0.02;    M=500;
φ = zeros(Float64,M,2);
φ[1,:] = xₑ;
for ii=2:M
    φ[ii,:] = φ[ii-1,:] + (-Fc(φ[ii-1,:])+Fg(φ[ii-1,:]))dt;
end

Sm = gMAM.action(res,F,G);
Ss = gMAM.action(flipdim(φ,1),F,G);
Su = Float64(subs(U,x[1]=>xₑ[1],x[2]=>xₑ[2])) - Float64(subs(U,x[1]=>x₀[1],x[2]=>x₀[2]));
@show(Sm);    @show(Ss);    @show(Su);

Sm = 0.5378858996315377
Ss = 0.5475973376844097
Su = 0.48471963032875465


In [43]:
mapPlt = plot(φ[:,1],φ[:,2], line=2, marker=0,label="SoS");
plot!(mapPlt, res[:,1],res[:,2], line=0, marker=3, label="MAP")
# plot!(mapPlt, xlims=(-1.05,0.05),ylims=(-0.05,1.05))
# Plots.svg("Path1.svg")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.8 
 
 
 -0.6 
 
 
 -0.4 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
<polyline clip-path="url(#clip2102)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 564.999,22.1145 562.042,39.0815 559.019,55.2037 555.931,70.5234 552.779,85.0805 549.564,98.913 546.286,112.057 542.946,124.547 539.544,136.416 536.081,147.694 
 532.557,158.412 528.973,168.596 525.33,178.274 521.627,187.471 517.865,196.21 514.044,204.516 510.166,212.409 506.23,219.909 502.237,227.038 498.188,233.812 
 494.083,240.25 489.923,246.369 485.709,252.184 481.44,257.711 477.119,262.964 472.746,267.957 468.321,272.702 463.845,277.212 459.321,281.499 454.748,285.574 
 450.127,289.448 445.461,293.129 440.75,296.629 435.996,299.957 431.2,303.119 426.363,306.126 421.488,308.985 416.575,311.703 411.627,314.287 406.645,316.743 
 401.632,319.079 396.588,321.3 391.518,323.412 386.421,325.421 381.302,327.331 376.161,329.147 371.002,330.874 365.827,332.517 360.638,334.08 355.437,335.566 
 350.229,336.98 345.014,338.325 339.796,339.605 334.578,340.822 329.363,341.98 324.152,343.082 318.95,344.131 313.758,345.129 308.581,346.079 303.42,346.983 
 298.279,347.843 293.161,348.662 288.068,349.442 283.003,350.184 277.969,350.891 272.969,351.563 268.006,352.204 263.082,352.814 258.2,353.396 253.363,353.949 
 248.572,354.477 243.831,354.979 239.142,355.458 234.506,355.914 229.927,356.348 225.406,356.763 220.945,357.157 216.546,357.534 212.211,357.893 207.941,358.235 
 203.737,358.561 199.602,358.872 195.537,359.169 191.541,359.452 187.618,359.721 183.767,359.979 179.989,360.225 176.285,360.459 172.656,360.683 169.102,360.896 
 165.622,361.1 162.219,361.295 158.89,361.481 155.637,361.658 152.46,361.827 149.357,361.989 146.33,362.144 143.377,362.291 140.497,362.432 137.691,362.567 
 134.958,362.696 132.297,362.819 129.707,362.936 127.188,363.049 124.737,363.156 122.356,363.259 120.042,363.357 117.794,363.451 115.612,363.541 113.494,363.627 
 111.439,363.709 109.445,363.787 107.513,363.862 105.64,363.934 103.825,364.003 102.067,364.069 100.364,364.132 98.7162,364.192 97.1212,364.25 95.578,364.305 
 94.0854,364.358 92.6419,364.408 91.2464,364.457 89.8976,364.503 88.5942,364.547 87.3351,364.59 86.1188,364.631 84.9444,364.669 83.8105,364.707 82.7159,364.742 
 81.6596,364.777 80.6403,364.809 79.657,364.841 78.7085,364.871 77.7938,364.9 76.9117,364.927 76.0614,364.954 75.2417,364.979 74.4517,365.003 73.6904,365.027 
 72.9568,365.049 72.2501,365.07 71.5694,365.091 70.9137,365.11 70.2822,365.129 69.6742,365.147 69.0887,365.164 68.5251,365.181 67.9825,365.197 67.4603,365.212 
 66.9576,365.226 66.4739,365.24 66.0084,365.254 65.5606,365.266 65.1297,365.279 64.7151,365.291 64.3164,365.302 63.9328,365.313 63.5639,365.323 63.209,365.333 
 62.8678,365.343 62.5397,365.352 62.2241,365.36 61.9207,365.369 61.629,365.377 61.3486,365.385 61.079,365.392 60.8198,365.399 60.5706,365.406 60.3312,365.412 
 60.101,365.419 59.8797,365.425 59.6671,365.43 59.4627,365.436 59.2663,365.441 59.0776,365.446 58.8962,365.451 58.7219,365.456 58.5544,365.46 58.3935,365.465 
 58.2389,365.469 58.0903,365.473 57.9476,365.477 57.8105,365.48 57.6787,365.484 57.5522,365.487 57.4306,365.49 57.3138,365.493 57.2015,365.496 57.0937,365.499 
 56.9902,365.502 56.8907,365.504 56.7952,365.507 56.7034,365.509 56.6152,365.511 56.5305,365.514 56.4492,365.516 56.3711,365.518 56.296,365.52 56.224,365.522 
 56.1548,365.523 56.0883,365.525 56.0244,365.527 55.9631,365.528 55.9042,365.53 55.8476,365.531 55.7933,365.533 55.7411,365.534 55.691,365.535 55.6429,365.536 
 55.5967,365.538 55.5523,365.539 55.5097,365.54 55.4688,365.541 55.4295,365.542 55.3918,365.543 55.3555,365.544 55.3207,365.545 55.2873,365.545 55.2552,365.546 
 55.2244,365.547 55.1948,365.548 55.1664,365.548 55.1391,365.549 55.1129,365.55 55.0877,365.55 55.0635,365.551 55.0403,365.552 55.0181,365.55

### Quasipotential comparison

We now plot a comparison bewteen the true and SOS computed quasipotential along a line.

In [47]:
γ = 1.0;    μ = 2.5;
@polyvar x[1:2]
F(X::Vector) = [X[1] - X[1]^3 - γ*X[1]X[2]^2;
                 -μ*(X[1]^2 + 1)X[2]];
f = F(x);
U = NormalSoS.normdecomp(f,x, MosekSolver(),2);
fg = -differentiate(U,x)
Fg(X::Vector) = [Float64(subs(fg[1],x[1]=>X[1], x[2]=>X[2]));
                 Float64(subs(fg[2],x[1]=>X[1], x[2]=>X[2]))];
fc = f + differentiate(U,x)
Fc(X::Vector) = [Float64(subs(fc[1],x[1]=>X[1], x[2]=>X[2]));
                 Float64(subs(fc[2],x[1]=>X[1], x[2]=>X[2]))];

Tspan = 10.0;  # Time span
N = 400;
x₀ = [-1.0 0.0];
dt = 0.02;    M=500;

xVec = collect(-1.5:0.1:0.0)
Smx = zeros(xVec);
Ssx = zeros(xVec);
for (idx,xi) in enumerate(xVec)
    xₑ = [xi 0];
    φ₀ = MAP.makepath(x₀,xₑ,N);
    for ii=1:N-2
        φ₀[ii,2] = 0.1*randn();
    end
    
    # Find the optimal path and true action
    resObj = MAP.optimalpath(F,G, x₀,xₑ, Tspan,N, φ₀);
    φ = [x₀; Optim.minimizer(resObj); xₑ];
    Smx[idx] = gMAM.action(φ,F,G);
    
    # Compute the SOS predicted action
    ψ = zeros(Float64,M,2);
    ψ[1,:] = xₑ;
    for ii=2:M
        ψ[ii,:] = ψ[ii-1,:] + (-Fc(ψ[ii-1,:])+Fg(ψ[ii-1,:]))dt;
    end
    Ssx[idx] = gMAM.action(flipdim(ψ,1),F,G);
end

xVec = collect(-1.5:0.1:1.5)
Smy = zeros(xVec);
Ssy = zeros(xVec);
for (idx,xi) in enumerate(xVec)
    xₑ = [-1.0 xi];
    φ₀ = MAP.makepath(x₀,xₑ,N);
    for ii=1:N-2
        φ₀[ii,2] = 0.1*randn();
    end
    
    # Find the optimal path and true action
    resObj = MAP.optimalpath(F,G, x₀,xₑ, Tspan,N, φ₀);
    φ = [x₀; Optim.minimizer(resObj); xₑ];
    Smy[idx] = gMAM.action(φ,F,G);
    
    # Compute the SOS predicted action
    ψ = zeros(Float64,M,2);
    ψ[1,:] = xₑ;
    for ii=2:M
        ψ[ii,:] = ψ[ii-1,:] + (-Fc(ψ[ii-1,:])+Fg(ψ[ii-1,:]))dt;
    end
    Ssy[idx] = gMAM.action(flipdim(ψ,1),F,G);
end

Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^4, x1^3x2, x1^2x2^2, x1^3, x1^2x2, x1x2^2, x1^2, x1x2, x2^2, x1, x2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 544             
  Cones                  : 0               
  Scalar variables       : 15              
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  

MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(544) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(545) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(546) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(548) of matrix 'A'.
MOSEK warning 705: #6 (nearly) zero elements are specified in sparse row ''(549) of matrix 'A'.
MOSEK warning 705: #5 (nearly) zero elements are specified in sparse row ''(550) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(551) of matrix 'A'.
MOSEK warning 705: #2 (nearly) zero elements are specified in sparse row ''(552) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(553) of matrix 'A'.
Warning number 705 is disabled.
Problem
  Name                   :                 
  Objective sense        : min             
  Type  

Optimisation for T=10.00 gives S=0.78
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.78
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.78
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.78
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.78
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.78
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.78
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.78
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.78
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.78
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=0.78
Optimisation not converged
Optimisation for T=10.00 gives S=0.46
Optimisation is not converged, rerunning...
Optimisation for T=10.00 gives S=

In [58]:
plt = plot(plot(ylim=(0.0,0.4)), plot(ylim=(0.0,4.0), legend=false), layout = @layout([a; b]), size=(400,700))

xVec = collect(-1.5:0.1:0.0)
Us = xi->Float64(subs(U,x[1]=>xi,x[2]=>x₀[2])) - Float64(subs(U,x[1]=>x₀[1],x[2]=>x₀[2]));
plot!(plt[1], collect(xVec[1]:0.01:xVec[end]),Us, label="U")
plot!(plt[1], xVec,Smx, line=0, marker=3, label="MAP")
plot!(plt[1], xVec,Ssx, line=0, marker=(3,:x), label="SOS")

xVec = collect(-1.5:0.1:1.5)
Us = xi->Float64(subs(U,x[1]=>x₀[1],x[2]=>xi)) - Float64(subs(U,x[1]=>x₀[1],x[2]=>x₀[2]));
plot!(plt[2], collect(xVec[1]:0.01:xVec[end]),Us, label="U")
plot!(plt[2], xVec,Smy, line=0, marker=(3), label="MAP")
plot!(plt[2], xVec,Ssy, line=0, marker=(3,:x), label="SOS")
Plots.svg("Bounds3")

## Zhou's quadratic system

In [3]:
## Example 1: Quadratic system from Zhou et al (2012)
@polyvar x[1:2]
F(X::Vector) = [-X[1] + 2.0X[2]^2;
                -1.5X[1]*X[2] - 2.0X[2]];
f = F(x);
@time U = NormalSoS.normdecomp(f,x, MosekSolver(),2);

fg = -differentiate(U,x)
Fg(X::Vector) = [Float64(subs(fg[1],x[1]=>X[1], x[2]=>X[2]));
                 Float64(subs(fg[2],x[1]=>X[1], x[2]=>X[2]))];
fc = f + differentiate(U,x)
Fc(X::Vector) = [Float64(subs(fc[1],x[1]=>X[1], x[2]=>X[2]));
                 Float64(subs(fc[2],x[1]=>X[1], x[2]=>X[2]))];

plt = NormalSoS.plotlandscape(f,U,x,([-3 3],[-3 3]),true);    plot(plt)
NormalSoS.checknorm(f,U,x)

Chosen basis as:
DynamicPolynomials.Monomial{true}[x1^2, x1x2, x2^2, x1, x2, 1]
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 101             
  Cones                  : 0               
  Scalar variables       : 8               
  Matrix variables       : 2               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 1
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective s

 22.478684 seconds (11.84 M allocations: 613.509 MiB, 1.89% gc time)


0.2735042736156235

In [4]:
# Obtain the minimum action path
x₀ = [0.0 0.0];  # Start point (row vector)
xₑ = [-1.0 0.8];
Tspan = 50.0;  # Time span
N = 400;
φ₀ = MAP.makepath(x₀,xₑ,N);
for ii=1:N-2
    φ₀[ii,2] = 0.1*randn();
end

σ = 1.0;
G(X::Vector) = σ*[1.0; 1.0];

# Use Optim to optimise over path φ
resObj = MAP.optimalpath(F,G, x₀,xₑ, Tspan,N, φ₀);
res = [x₀; Optim.minimizer(resObj); xₑ];

Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation is not converged, rerunning...
Optimisation for T=50.00 gives S=2.09
Optimisation not converged


In [5]:
# Find the SOS-predicted minimum action path and compare ΔU with gMAM
dt = 0.02;    M=500;
φ = zeros(Float64,M,2);
φ[1,:] = xₑ;
for ii=2:M
    φ[ii,:] = φ[ii-1,:] + (-Fc(φ[ii-1,:])+Fg(φ[ii-1,:]))dt;
    # println((Fc(φ[ii,:])-Fg(φ[ii,:]))dt)
end

Sm = gMAM.action(res,F,G);
Ss = gMAM.action(flipdim(φ,1),F,G);
Su = Float64(subs(U,x[1]=>φ[1,1],x[2]=>φ[1,2])) - Float64(subs(U,x[1]=>φ[end,1],x[2]=>φ[end,2]));
@show(Sm);    @show(Ss);    @show(Su);

Sm = 1.0488605186535092
Ss = 1.0493808409240384
Su = 0.9266666564578752


In [6]:
mapPlt = plot(φ[:,1],φ[:,2], line=2, marker=0,label="SoS");
plot!(mapPlt, res[:,1],res[:,2], line=2, marker=0, label="MAP")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.00 
 
 
 -0.75 
 
 
 -0.50 
 
 
 -0.25 
 
 
 0.00 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
<polyline clip-path="url(#clip1402)" style="stroke:#009af9; stroke-width:2; stroke-opacity:1; fill:none" points="
 53.1849,22.1145 50.3198,36.9974 48.6225,51.2905 47.9794,64.9932 48.2847,78.1097 49.4402,90.6487 51.3556,102.622 53.9478,114.045 57.1406,124.932 60.8646,135.304 
 65.0563,145.178 69.6584,154.575 74.6184,163.514 79.8893,172.016 85.4281,180.1 91.1964,187.786 97.1594,195.094 103.286,202.041 109.547,208.647 115.918,214.928 
 122.376,220.901 128.901,226.583 135.475,231.989 142.08,237.133 148.703,242.029 155.331,246.69 161.951,251.129 168.555,255.358 175.131,259.388 181.673,263.23 
 188.173,266.894 194.625,270.388 201.022,273.723 207.362,276.907 213.638,279.947 219.847,282.852 225.987,285.628 232.054,288.282 238.046,290.82 243.961,293.249 
 249.797,295.574 255.553,297.8 261.229,299.932 266.822,301.976 272.333,303.934 277.76,305.813 283.105,307.616 288.366,309.346 293.544,311.006 298.639,312.602 
 303.651,314.134 308.58,315.608 313.428,317.024 318.194,318.387 322.879,319.698 327.484,320.961 332.01,322.176 336.457,323.347 340.826,324.475 345.118,325.563 
 349.334,326.611 353.475,327.623 357.542,328.599 361.536,329.541 365.457,330.451 369.307,331.329 373.087,332.178 376.798,332.999 380.44,333.792 384.016,334.559 
 387.525,335.301 390.969,336.02 394.349,336.715 397.666,337.388 400.92,338.04 404.114,338.671 407.248,339.283 410.322,339.876 413.339,340.452 416.298,341.009 
 419.201,341.551 422.049,342.076 424.843,342.586 427.584,343.081 430.272,343.561 432.909,344.028 435.496,344.482 438.033,344.923 440.521,345.352 442.961,345.769 
 445.355,346.174 447.702,346.568 450.004,346.952 452.262,347.326 454.476,347.689 456.647,348.043 458.776,348.388 460.864,348.724 462.912,349.051 464.919,349.37 
 466.888,349.681 468.819,349.984 470.711,350.279 472.567,350.567 474.387,350.848 476.172,351.123 477.921,351.39 479.637,351.651 481.319,351.906 482.968,352.155 
 484.585,352.399 486.17,352.636 487.724,352.868 489.248,353.095 490.742,353.316 492.207,353.533 493.643,353.744 495.051,353.951 496.431,354.154 497.784,354.352 
 499.11,354.545 500.411,354.735 501.686,354.92 502.936,355.101 504.161,355.279 505.362,355.452 506.54,355.623 507.694,355.789 508.826,355.952 509.936,356.112 
 511.023,356.268 512.089,356.422 513.134,356.572 514.159,356.719 515.163,356.863 516.148,357.005 517.113,357.143 518.059,357.279 518.986,357.412 519.895,357.543 
 520.787,357.671 521.66,357.797 522.516,357.92 523.356,358.041 524.179,358.159 524.985,358.276 525.776,358.39 526.551,358.502 527.31,358.612 528.055,358.72 
 528.785,358.826 529.5,358.93 530.202,359.032 530.889,359.133 531.563,359.231 532.223,359.328 532.871,359.423 533.505,359.517 534.127,359.608 534.737,359.698 
 535.335,359.787 535.921,359.874 536.495,359.96 537.058,360.044 537.609,360.126 538.15,360.207 538.68,360.287 539.199,360.366 539.709,360.443 540.208,360.519 
 540.697,360.593 541.177,360.667 541.647,360.739 542.107,360.81 542.559,360.879 543.001,360.948 543.435,361.015 543.86,361.082 544.277,361.147 544.686,361.211 
 545.086,361.274 545.478,361.337 545.863,361.398 546.24,361.458 546.61,361.517 546.972,361.575 547.327,361.633 547.675,361.689 548.016,361.745 548.35,361.8 
 548.678,361.853 548.999,361.906 549.314,361.959 549.622,362.01 549.924,362.061 550.221,362.11 550.511,362.159 550.796,362.208 551.075,362.255 551.348,362.302 
 551.616,362.348 551.879,362.393 552.137,362.438 552.389,362.482 552.636,362.525 552.879,362.568 553.116,362.61 553.349,362.652 553.578,362.692 553.801,362.733 
 554.021,362.772 554.236,362.811 554.446,362.85 554.653,362.888 554.855,362.925 555.053,362.962 555.248,362.998 555.438,363.033 555.625,363.069 555.808,363.103 
 555.987,363.137 556.163,363.171 556.335,363.204 556.504,363.237 556.67,363.269 556.832,363.301 556.991,363.332 557.147,363.363 557.3,363

### Quasipotential comparison

We now plot a comparison bewteen the true and SOS computed quasipotential along a line.

In [17]:
Tspan = 50.0;  # Time span
N = 400;
xVec = collect(0.0:0.5:4.0);
Sm = zeros(xVec);

x₀ = [0.0 0.0];
for (idx,xi) in enumerate(xVec)
    xₑ = [xi 0.0];
    φ₀ = MAP.makepath(x₀,xₑ,N);
    
    # Find the optimal path
    resObj = MAP.optimalpath(F,G, x₀,xₑ, Tspan,N, φ₀);
    φ = [x₀; Optim.minimizer(resObj); xₑ];
    
    # Compute the true action
    Sm[idx] = gMAM.action(φ,F,G);
    # Su[idx] = Float64(subs(U,x[1]=>φ[end,1],x[2]=>φ[end,2])) - Float64(subs(U,x[1]=>φ[1,1],x[2]=>φ[1,2])); 
end
Sm[1] = 0.0;

Us = xi->Float64(subs(U,x[1]=>xi,x[2]=>x₀[2])) - Float64(subs(U,x[1]=>x₀[1],x[2]=>x₀[2]));
plot(collect(xVec[1]:0.01:xVec[end]),Us, label="MAP")
plot!(xVec,Sm, line=0, marker=2, label="SoS")

Optimisation for T=50.00 gives S=0.00
Optimisation is converged.
Optimisation for T=50.00 gives S=0.25
Optimisation is converged.
Optimisation for T=50.00 gives S=1.00
Optimisation is converged.
Optimisation for T=50.00 gives S=2.25
Optimisation is converged.
Optimisation for T=50.00 gives S=4.00
Optimisation is converged.
Optimisation for T=50.00 gives S=6.25
Optimisation is converged.
Optimisation for T=50.00 gives S=9.00
Optimisation is converged.
Optimisation for T=50.00 gives S=12.25
Optimisation is converged.
Optimisation for T=50.00 gives S=16.00
Optimisation is converged.


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 8 
 
<polyline clip-path="url(#clip2402)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 38.2276,365.565 39.5433,365.563 40.8591,365.557 42.1748,365.546 43.4906,365.531 44.8063,365.511 46.122,365.488 47.4378,365.46 48.7535,365.428 50.0693,365.391 
 51.385,365.351 52.7008,365.305 54.0165,365.256 55.3323,365.202 56.648,365.144 57.9638,365.082 59.2795,365.016 60.5953,364.945 61.911,364.87 63.2267,364.79 
 64.5425,364.707 65.8582,364.619 67.174,364.526 68.4897,364.43 69.8055,364.329 71.1212,364.224 72.437,364.114 73.7527,364 75.0685,363.882 76.3842,363.76 
 77.7,363.633 79.0157,363.502 80.3314,363.367 81.6472,363.228 82.9629,363.084 84.2787,362.936 85.5944,362.783 86.9102,362.627 88.2259,362.466 89.5417,362.3 
 90.8574,362.131 92.1732,361.957 93.4889,361.779 94.8047,361.596 96.1204,361.409 97.4361,361.218 98.7519,361.023 100.068,360.823 101.383,360.619 102.699,360.411 
 104.015,360.199 105.331,359.982 106.646,359.761 107.962,359.535 109.278,359.306 110.594,359.072 111.909,358.834 113.225,358.591 114.541,358.344 115.857,358.093 
 117.172,357.838 118.488,357.578 119.804,357.314 121.12,357.045 122.435,356.773 123.751,356.496 125.067,356.215 126.383,355.929 127.698,355.639 129.014,355.345 
 130.33,355.047 131.646,354.744 132.961,354.437 134.277,354.126 135.593,353.811 136.909,353.491 138.224,353.167 139.54,352.838 140.856,352.505 142.172,352.168 
 143.487,351.827 144.803,351.482 146.119,351.132 147.434,350.777 148.75,350.419 150.066,350.056 151.382,349.689 152.697,349.318 154.013,348.942 155.329,348.562 
 156.645,348.178 157.96,347.789 159.276,347.397 160.592,347 161.908,346.598 163.223,346.192 164.539,345.782 165.855,345.368 167.171,344.95 168.486,344.527 
 169.802,344.099 171.118,343.668 172.434,343.232 173.749,342.792 175.065,342.348 176.381,341.899 177.697,341.446 179.012,340.989 180.328,340.528 181.644,340.062 
 182.96,339.592 184.275,339.117 185.591,338.639 186.907,338.156 188.223,337.668 189.538,337.177 190.854,336.681 192.17,336.181 193.486,335.676 194.801,335.168 
 196.117,334.655 197.433,334.137 198.749,333.616 200.064,333.09 201.38,332.56 202.696,332.025 204.012,331.486 205.327,330.943 206.643,330.396 207.959,329.844 
 209.275,329.288 210.59,328.728 211.906,328.163 213.222,327.595 214.538,327.021 215.853,326.444 217.169,325.862 218.485,325.276 219.801,324.686 221.116,324.091 
 222.432,323.492 223.748,322.889 225.064,322.282 226.379,321.67 227.695,321.054 229.011,320.434 230.326,319.809 231.642,319.18 232.958,318.547 234.274,317.909 
 235.589,317.267 236.905,316.621 238.221,315.971 239.537,315.316 240.852,314.657 242.168,313.994 243.484,313.326 244.8,312.654 246.115,311.978 247.431,311.298 
 248.747,310.613 250.063,309.924 251.378,309.231 252.694,308.533 254.01,307.831 255.326,307.125 256.641,306.414 257.957,305.7 259.273,304.98 260.589,304.257 
 261.904,303.529 263.22,302.797 264.536,302.061 265.852,301.321 267.167,300.576 268.483,299.827 269.799,299.073 271.115,298.315 272.43,297.553 273.746,296.787 
 275.062,296.016 276.378,295.242 277.693,294.462 279.009,293.679 280.325,292.891 281.641,292.099 282.956,291.303 284.272,290.502 285.588,289.697 286.904,288.888 
 288.219,288.074 289.535,287.256 290.851,286.434 292.167,285.608 293.482,284.777 294.798,283.942 296.114,283.103 297.43,282.259 298.745,281.411 300.061,280.559 
 301.377,279.703 302.693,278.842 304.008,277.977 305.324,277.107 306.64,276.234 307.956,275.356 309.271,274.473 310.587,273.587 311.903,272.696 313.219,271.801 
 314.534,270.902 315.85,269.998 317.166,269.09 318.481,268.178 319.797,267.261 321.113,266.34 322.429,265.415 323.744,264.485 325.06,263.552 326.376,262.614 
 327.692,261.671 329.007,260.725 330.323,259.774 331.639,258.819 332.955,257.859 334.27,256.895 335.586,255.927 336.902,254.955 338.218,253.978 339.533,252.997 
 340.849,252.0

In [39]:
xVec = -collect(0.0:0.24:1.6)

7-element Array{Float64,1}:
 -0.0 
 -0.24
 -0.48
 -0.72
 -0.96
 -1.2 
 -1.44